# Importing data
This notebook demonstrates how Python, using the `requests`, `io`, and `pandas` libraries, can generate a working dataset from on-line resources. Specifically, this notebook forms a request to a web service, here the NWIS API, and handles the response in such a way as to form a tidy data frame, which is saved as a comma separated value (CSV) formatted file on the local disk. 

Service Documentation
https://waterservices.usgs.gov/rest/DV-Test-Tool.html

Example request in URL format:<br>
http://waterservices.usgs.gov/nwis/dv/?format=rdb&sites=02089000&startDT=2010-10-01&endDT=2017-09-30&statCd=00003&parameterCd=00060&siteStatus=all

#### Import the Python libraries required to run the script

In [1]:
#Import libraries
import requests
import pandas as pd
import io

#### Assemble the parameters that will be used in the data request

In [2]:
#Set site, parameter, and stat codes
siteNo = '02087500' #Neuse R. Near Clayton 
pcode = '00060'     #Discharge (cfs)
scode = '00003'     #Daily mean

In [3]:
#Set start and end dates
startDate = '1930-10-01'
endDate = '2017-09-03'

#### Format the request, then send it and store the response as a variable

In [4]:
#Construct the service URL and parameters
#https://waterdata.usgs.gov/nwis/dv?
url =  'https://waterservices.usgs.gov/nwis/dv'
params = {'sites':siteNo,
          'parameterCd':pcode,
          'statCd':scode,
          'startDT':startDate,
          'endDT':endDate,
          'format':'rdb',
          'siteStatus':'all'
         }

In [5]:
#Send the requests and translate the response
response_raw = requests.get(url,params)
response_clean = response_raw.content.decode('utf-8')

#### Clean up the response and read it into a data frame

In [6]:
#Create a list of metadata rows to skip; rows 1-29 and 31 
rowsToSkip = list(range(28))
#Append '30' to the list
rowsToSkip.append(29)

In [7]:
#Convert the data into a data frame
df = pd.read_csv(io.StringIO(response_clean),
                 skiprows=rowsToSkip,     #Skip metadta and data spec lines
                 delimiter='\t',          #Set to tab delimited
                 dtype={'site_no':'str'}) #Set site_no to a string datatype

#### Examine the resulting data frame

In [8]:
#Display the first 5 rows
df.head()

,agency_cd,site_no,datetime,85235_00060_00003,85235_00060_00003_cd
0,USGS,02087500,1930-10-01,347.0,A
1,USGS,02087500,1930-10-02,173.0,A
2,USGS,02087500,1930-10-03,132.0,A
3,USGS,02087500,1930-10-04,125.0,A
4,USGS,02087500,1930-10-05,125.0,A


In [9]:
#Rename the last two fields
df.rename(columns={'85235_00060_00003':'MeanFlow_cfs','85235_00060_00003_cd':'Confidence'},inplace=True)
df.head()

,agency_cd,site_no,datetime,MeanFlow_cfs,Confidence
0,USGS,02087500,1930-10-01,347.0,A
1,USGS,02087500,1930-10-02,173.0,A
2,USGS,02087500,1930-10-03,132.0,A
3,USGS,02087500,1930-10-04,125.0,A
4,USGS,02087500,1930-10-05,125.0,A


In [10]:
df.dtypes

agency_cd        object
site_no          object
datetime         object
MeanFlow_cfs    float64
Confidence       object
dtype: object

#### Save the dataframe to a csv file

In [11]:
#Save to a csv file
df.to_csv('GageData.csv',index=False)